In [ ]:
setwd("/Output")
library(reshape2)
library(stringr)
library(tidyverse)
library(dplyr)
library(data.table)

In [ ]:
parent.folder<-"T:/PIgroup Nael NadifKasri/Sofia/2025/MEA_seq_patient/Code/Data/MEA_data"

In [ ]:
all_summary_files <- list.files(parent.folder, full.names = T, recursive = T, pattern = "Well_Summary_Parameters_hfNB_s150.csv")#list.files( full.names = F, recursive = T, pattern = ".*PT_all*")

#only if you need to subset for specific files, if you have run 1.Automatic_burst_detection_MCS_axion multiple times with different settings 
#toMatch <- c("2024_v4_results_hf_s150prop0.4_freqNB9_in_0.25","2024_v4_results_nohf_s150prop0.4_freqNB9_in_0.25")
#matches <- unique (grep(paste(toMatch,collapse="|"), 
#                        all_summary_files, value=TRUE))

all_summary_files<-all_summary_files[all_summary_files %in% matches]
all_summary_files <- lapply(all_summary_files,read.csv)
all_summary_files<-do.call(rbind, all_summary_files)
all_summary_files<-all_summary_files[-1]


In [ ]:
cell_line_overview<-read.csv("T:/PIgroup Nael NadifKasri/Patient paper/2025/Data/Cell_line_overview.csv") #drav cell lines are norm to WTC

In [ ]:
cell_line_overview<-read.csv("T:/PIgroup Nael NadifKasri/Sofia/2025/MEA_seq_patient/Code/Data/cell_lines/Cell_line_overview.csv")

In [ ]:
full_results<-dplyr::left_join(all_summary_files, cell_line_overview, by=c("Phenotype" = "Cell_line_name"))
full_results<-dplyr::left_join(all_summary_files, cell_line_overview, by=c("Phenotype" = "Cell_line_name"))
exclude_cell_lines<-rownames(full_results[full_results$Phenotype=="EXCLUDE",]) #if you need to exclude cell lines

full_results<-full_results[!rownames(full_results) %in% exclude_cell_lines,]
exclude_cell_lines<-rownames(full_results[full_results$Phenotype=="ChaserrKO",]) #not included
full_results<-full_results[!rownames(full_results) %in% exclude_cell_lines,]

full_results<-full_results[!full_results$Remove=="yes",]
full_results<-droplevels(full_results)
levels(factor(full_results$Phenotype))
length(levels(factor(full_results$Phenotype)))

In [ ]:
full_results <- full_results %>%
  mutate(Phenotype = ifelse(Phenotype == "EHMT_CR", "EHMT1_CR", Phenotype)) #consistent naming


In [ ]:
levels(factor(full_results$Phenotype))
length(levels(factor(full_results$Phenotype)))

In [ ]:
full_results$numb_electrodes<-ifelse(full_results$machine=="mcs",12,16)
full_results$hf_Network_Burst_Percentage[is.na(full_results$hf_Network_Burst_Percentage)] <- 0
full_results$hf_and_NB_Percentage<-full_results$hf_Network_Burst_Percentage+full_results$Network_Burst_Percentage
full_results$hf_Network_Burst_Frequency[is.na(full_results$hf_Network_Burst_Frequency)] <- 0
full_results$hf_and_NB_rate<-full_results$hf_Network_Burst_Frequency+full_results$Network_Burst_Frequency
all_registered_variables<-colnames(full_results)
hf_variables<-grep("*hf", all_registered_variables, value = TRUE)#all_registered_variables[grepl("*hf", all_registered_variables)]
hf_variables<-hf_variables[-c(2,10,11)]#define which ones o keep. 
full_results<-full_results[,!colnames(full_results) %in% hf_variables]


In [ ]:
weird_or_same<-c("Random_Spikes_Percentage_Avg","Spike_Frequency_per_Electrode",
         "total_spikes_in_all_NBs_d_nNB","numb_electrodes","propr_active_electrodes","Network_Burst_Frequency")#removing redundant or un need output from 1.Automatic_burst_detection_MCS_axion

full_results<-full_results[,!colnames(full_results) %in% weird_or_same]
no_NB<-rownames(full_results[full_results$Number_of_Bursts<=1 |full_results$Number_of_Bursting_Electrodes<=1 |full_results$Number_of_Network_Bursts<=2|is.na(full_results$Number_of_Network_Bursts),])

no_NB<-full_results[no_NB,]
no_NB<-droplevels(no_NB)
write.csv(no_NB,"wells_with_no_bursting_NB_activity.csv")
dep_on_recording_time<-c("Number_of_spikes","Number_of_Bursting_Electrodes",
                         "Number_of_Bursts","Number_of_Network_Bursts")
n_ID<-c(1:dim(full_results)[1])
full_results$Unique_ID<-paste0(full_results$PT_all_path,"_",full_results$Well_Label,"_",n_ID)
full_results$PT_all_path_Well_Label<-paste0(full_results$PT_all_path,"_",full_results$Well_Label)

full_results_NB<-full_results[!rownames(full_results) %in% rownames(no_NB),!colnames(full_results) %in% dep_on_recording_time]
full_results<-full_results[,!colnames(full_results) %in% dep_on_recording_time]

In [ ]:
colnames(full_results_NB)
length(colnames(full_results_NB))

In [ ]:
short_names<-c('Well_Label','Phenotype','Firing_rate_Hz','EB_Dur','EB_Dur_Std','Spikes_per_EB',
  'Spikes_per_EB_Std','EB_ISI','EB_ISI_Std','IEBI','IEBI_Std','IEBI_CV','IEBI_CV_Std',
  'EB_rate_Hz','EB_rate_Std','EB_prop','EB_prop_Std',
  'hfNB_rate','NB_prop',
  'NB_dur','NB_dur_Std','Firing_Rate_in_NB_Hz', 'Firing_Rate_in_NB_Std','INBI','INBI_Std',
  'batch','cell_line_batch','machine','PT_all_path','DIV_range','Syndrome','Gene','Identidy','Own_control',
   'Remove','isogenic','Remarks',
  'Pool_Kleefstra','Pool_WTC_CHD2','Pool_CACNA1A','Pool_SMARCB1','Pool_ARID1B_2_rescue','Pool_dravet',
   'hfNB_and_NB_prop','hf_and_NB_rate','Unique_ID','PT_all_path_Well_Label')#re-naming
length(short_names)

In [ ]:
colnames(full_results_NB)<-short_names
colnames(full_results)<-short_names

In [ ]:
full_results<-droplevels(full_results)
full_results_NB<-droplevels(full_results_NB)
full_results_NB[apply(is.na(full_results_NB[, sapply(full_results_NB, is.numeric)]), 1, any), ]

In [ ]:
DIV49_sub<-levels(factor(full_results_NB[full_results_NB$DIV_range=="DIV49","Phenotype"]))
dev_sub<-full_results_NB %>%
  filter(DIV_range %in% c("DIV28", "DIV35", "DIV42", "DIV49")) %>%
  group_by(Phenotype) %>%
  filter(all(c("DIV28", "DIV35", "DIV42", "DIV49") %in% DIV_range)) %>%
  summarise() %>%
  ungroup()
dev_sub<-dev_sub$Phenotype #annotate if we have until DIV49 data

In [ ]:
full_results_NB$DIV49_sub<-ifelse(full_results_NB$Phenotype %in% DIV49_sub, "yes","no")
full_results_NB$dev_sub<-ifelse(full_results_NB$Phenotype %in% dev_sub, "yes","no")

In [ ]:
full_results$DIV49_sub<-ifelse(full_results$Phenotype %in% DIV49_sub, "yes","no")
full_results$dev_sub<-ifelse(full_results$Phenotype %in% dev_sub, "yes","no")

In [ ]:
setdiff(DIV49_sub,dev_sub)# find that aptient!
#Interestigly!! CHD2_PAT002 only shows NB at div 49!, that is whyt it is not in the developmental data!!

In [ ]:
write.csv(full_results,"Well_summary_parameters_all_hf_043_06_2025.csv")
write.csv(full_results_NB,"Well_summary_parameters_NB_hf_043_06_2025.csv")

In [ ]:
getwd()